<a href="https://colab.research.google.com/github/AgileDataScientist/absent-data-hospital-wait-time/blob/main/NYCBusDelayTextFieldClean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!cp /content/drive/My Drive/DataAnalystPortfolio/DataAnalysisNYCSchoolBusDelayTime/bus.tar.gz /content/dataset
!pip install pyjanitor
!pip install jupyter-dash
!pip install plotly

# **NYC School Bus Delay Data**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Plotly and Dash imports
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash.dependencies

# Python dplyr
from janitor import clean_names

# nltk imports
import nltk
import re
import string
from pprint import pprint

from google.colab import drive
drive.mount('/content/myDrive')

Drive already mounted at /content/myDrive; to attempt to forcibly remount, call drive.mount("/content/myDrive", force_remount=True).


## **Read Data**

In [4]:
onUpload = False
if onUpload:
  bus_data = pd.read_csv('/content/myDrive/MyDrive/Data Analyst Portfolio/Data Analysis NYC School Bus Delay Time/Bus_Breakdown_and_Delays.csv') #, index_col='Occurred_On', parse_dates=True)
else:  
  bus_data = pd.read_csv('/content/sample_data/Bus_Breakdown_and_Delays.csv') #, index_col='Occurred_On', parse_dates=True)

bus_data.head()  

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,School_Year,Busbreakdown_ID,Run_Type,Bus_No,Route_Number,Reason,Schools_Serviced,Occurred_On,Created_On,Boro,...,How_Long_Delayed,Number_Of_Students_On_The_Bus,Has_Contractor_Notified_Schools,Has_Contractor_Notified_Parents,Have_You_Alerted_OPT,Informed_On,Incident_Number,Last_Updated_On,Breakdown_or_Running_Late,School_Age_or_PreK
0,2015-2016,1227538,Special Ed AM Run,2621,J711,Heavy Traffic,75003,11/05/2015 08:10:00 AM,11/05/2015 08:12:00 AM,New Jersey,...,NaN,11,Yes,No,Yes,11/05/2015 08:12:00 AM,NaN,11/05/2015 08:12:14 AM,Running Late,School-Age
1,2015-2016,1227539,Special Ed AM Run,1260,M351,Heavy Traffic,06716,11/05/2015 08:10:00 AM,11/05/2015 08:12:00 AM,Manhattan,...,20MNS,2,Yes,Yes,No,11/05/2015 08:12:00 AM,NaN,11/05/2015 08:13:34 AM,Running Late,School-Age
2,2015-2016,1227540,Pre-K/EI,418,3,Heavy Traffic,C445,11/05/2015 08:09:00 AM,11/05/2015 08:13:00 AM,Bronx,...,15MIN,8,Yes,Yes,Yes,11/05/2015 08:13:00 AM,NaN,11/05/2015 08:13:22 AM,Running Late,Pre-K
3,2015-2016,1227541,Special Ed AM Run,4522,M271,Heavy Traffic,02699,11/05/2015 08:12:00 AM,11/05/2015 08:14:00 AM,Manhattan,...,15 MIN,6,No,No,No,11/05/2015 08:14:00 AM,NaN,11/05/2015 08:14:04 AM,Running Late,School-Age
4,2015-2016,1227542,Special Ed AM Run,3124,M373,Heavy Traffic,02116,11/05/2015 08:13:00 AM,11/05/2015 08:14:00 AM,Manhattan,...,NaN,6,No,No,No,11/05/2015 08:14:00 AM,NaN,11/05/2015 08:14:08 AM,Running Late,School-Age


In [5]:
# Clean up feature names
bus_data = clean_names(bus_data) 

In [6]:
bus_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529695 entries, 0 to 529694
Data columns (total 21 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   school_year                      529695 non-null  object
 1   busbreakdown_id                  529695 non-null  int64 
 2   run_type                         503672 non-null  object
 3   bus_no                           529682 non-null  object
 4   route_number                     529688 non-null  object
 5   reason                           529693 non-null  object
 6   schools_serviced                 529688 non-null  object
 7   occurred_on                      529695 non-null  object
 8   created_on                       529695 non-null  object
 9   boro                             518481 non-null  object
 10  bus_company_name                 529695 non-null  object
 11  how_long_delayed                 476328 non-null  object
 12  number_of_studen

In [7]:
# Missing data
bus_data.isnull().sum()

school_year                             0
busbreakdown_id                         0
run_type                            26023
bus_no                                 13
route_number                            7
reason                                  2
schools_serviced                        7
occurred_on                             0
created_on                              0
boro                                11214
bus_company_name                        0
how_long_delayed                    53367
number_of_students_on_the_bus           0
has_contractor_notified_schools         0
has_contractor_notified_parents         0
have_you_alerted_opt                    0
informed_on                             0
incident_number                    518825
last_updated_on                         0
breakdown_or_running_late               0
school_age_or_prek                      0
dtype: int64

In [8]:
# Unique values feature
bus_data['how_long_delayed'].nunique()

1877

## **HOW_LONG_DELAYED CLEAN**

The "how_long_delayed" feature is a mixture of numbers and text that needs to be cleaned to contain the number of minutes delayed. 

In [9]:
# Number of values to be cleaned
bus_data['busbreakdown_id'].nunique() -  bus_data['how_long_delayed'].isnull().sum()

476328

In [10]:
# Number of missing
bus_data['how_long_delayed'].isnull().sum()

53367

In [87]:
delay_df = pd.DataFrame(bus_data['how_long_delayed'])
delay_df['lowercase'] = delay_df['how_long_delayed'].fillna('0 min').str.lower()
delay_df[:5]

,how_long_delayed,lowercase
0,NaN,0 min
1,20MNS,20mns
2,15MIN,15min
3,15 MIN,15 min
4,NaN,0 min


In [115]:
index_keys = delay_df['lowercase'].value_counts().index
index_keys.value_counts()

31-45 min     1
30 min0       1
45 am         1
1346          1
35 mmin       1
             ..
30min/45mi    1
30-45mt       1
31 minutes    1
5/6  mins     1
45 miniute    1
Length: 1453, dtype: int64

In [142]:
index_clean = pd.DataFrame(index_keys)
index_clean[0].str.replace('mins', ' min ').str.replace('minutes',' min ')

0        31-45 min
1        16-30 min
2        46-60 min
3            0 min
4         0-15 min
           ...    
1448      45 min s
1449        ni0933
1450         6 mis
1451    30 m min .
1452    45 miniute
Name: 0, Length: 1453, dtype: object

In [73]:
delay_df['lowercase'].value_counts()

31-45  min     116145
16-30  min     107832
46-60  min      62185
0  min          53367
0-15  min       38306
                ...  
45  min s           1
ni0933              1
6 mis               1
30 m mins.          1
45  miniute         1
Name: lowercase, Length: 1453, dtype: int64

In [30]:
delay_df['lowercase_m'] = delay_df['lowercase'].str.extract('([0-9][0-9]+[mn]|[0-9][0-9]-[0-9][0-9][m]+|[0-9][0-9][minutes]+)', expand=False)
#.extract('([0-9][0-9]-[0-9] hour+|[0-9].[0-9] hour+|[0-9] [0-9]/[0-9] hour+|[0-9]/[0-9] hour+|[0-9] hour+)', expand=False)
delay_df.groupby(['lowercase'])['lowercase_m'].value_counts()

lowercase   lowercase_m
-10mins     10m             1
1 30mnts    30m             1
1 hour 15m  15m             3
1 hr 15min  15m             1
1 hr 30min  30m            30
                           ..
est. 35min  35m             1
est.15-20m  15-20m          1
est.20-30m  20-30m          1
est.35min   35m             1
ihr40min    40m             1
Name: lowercase_m, Length: 363, dtype: int64

In [ ]:
delay_df['text_portion'] = delay_df['lowercase'].str.extract('([0-9][0-9]-[0-9] hour+|[0-9].[0-9] hour+|[0-9] [0-9]/[0-9] hour+|[0-9]/[0-9] hour+|[0-9] hour+)', expand=False)
delay_df.groupby(['text_portion','lowercase'])['lowercase'].value_counts()

In [ ]:
delay_df.groupby(['text_portion'])['lowercase'].value_counts().index

In [111]:
hour_min = {'1:00 hour' : '60 min', 
            '1 1/2 hour': '90 min',
            '1 hour' :    '60 min',
            '45 -1 hour': '45-60 min',
            '1 hours':    '60 min',
            '1 hour 15m': '75 min',
            '1 hour?' :   '60 min',
            '1 hour 15':  '75 min',
            '1 hour lat': '60 min',
            '1 hour1/2':  '90 min',
            '1 hourhalf': '90 min',
            '1 hourr':    '60 min',
            '1.5 hour':   '90 min',
            '1/2 hour':   '30 min',
            '1/2 hour?':  '30 min',
            '2 hours':    '120 min',
            '2 hour':     '120 min',
            '2 hours tr': '120 min',
            '3 hour':     '180 min',
            '4 hours':    '240 min',
            '40-1 hour':  '40-60 min',
            '50-1 hour':  '50-60 min',
}

delay_df['delayed_min_text'] = delay_df['lowercase'].map(hour_min)
delay_df.groupby(['text_portion','lowercase'])['delayed_min_text'].value_counts()

text_portion  lowercase   delayed_min_text
0 hour        1:00 hour   60 min                 1
1 1/2 hour    1 1/2 hour  90 min                 2
1 hour        1 hour      60 min              1423
              1 hour 15   75 min                 1
              1 hour 15m  75 min                 3
              1 hour lat  60 min                 1
              1 hour1/2   90 min                 1
              1 hour?     60 min                 2
              1 hourhalf  90 min                 1
              1 hours     60 min                 4
              45 -1 hour  45-60 min              5
1 hourr       1 hourr     60 min                 1
1.5 hour      1.5 hour    90 min                10
1/2 hour      1/2 hour    30 min               135
              1/2 hour?   30 min                15
2 hour        2 hour      120 min                5
              2 hours     120 min               24
              2 hours tr  120 min                1
3 hour        3 hour      180 min      

In [138]:
delay_df['lowercase_x'] = delay_df['lowercase'].replace('m', ' m')
#delay_df[500005:500015]
delay_df

,how_long_delayed,lowercase,text_portion,delayed_min,delayed_min_text,lowercase_x
0,NaN,na,NaN,NaN,NaN,na
1,20MNS,20mns,20mns,NaN,NaN,20mns
2,15MIN,15min,15min,NaN,NaN,15min
3,15 MIN,15 min,15 min,NaN,NaN,15 min
4,NaN,na,NaN,NaN,NaN,na
...,...,...,...,...,...,...
529690,0-15 Min,0-15 min,0-15 min,NaN,NaN,0-15 min
529691,16-30 Min,16-30 min,16-30 min,NaN,NaN,16-30 min
529692,NaN,na,NaN,NaN,NaN,na
529693,61-90 Min,61-90 min,61-90 min,NaN,NaN,61-90 min


In [134]:
delay_df['text_portion'] = delay_df['lowercase'].str.extract('([0-9]*-[0-9]* min|[0-9]* min|[0-9]+[a-z]+)', expand=False)
#delay_df[500005:500015]
delay_df

,how_long_delayed,lowercase,text_portion,delayed_min,delayed_min_text
0,NaN,na,NaN,NaN,NaN
1,20MNS,20mns,20mns,NaN,NaN
2,15MIN,15min,15min,NaN,NaN
3,15 MIN,15 min,15 min,NaN,NaN
4,NaN,na,NaN,NaN,NaN


In [ ]:
delay_df['text_portion'].value_counts().index

In [47]:
time_unit_map = {}
for key in delay_df['text_portion'].value_counts().index :
  if key[0] == 'm':
    time_unit_map[key] = 'minutes'
  elif key[0] == 'h':  
    time_unit_map[key] = 'hours'

#time_unit_map

In [48]:
delay_df['time_unit_mapped'] = delay_df['text_portion'].map(time_unit_map)
delay_df['time_unit_mapped'].value_counts()

minutes    515717
hours        4175
Name: time_unit_mapped, dtype: int64

In [54]:
delay_df[delay_df['time_unit_mapped'] == 'hours'].groupby(['time_unit_mapped'])['lowercase'].value_counts()

time_unit_mapped  lowercase 
hours             1 hour        1423
                  1 hr          1349
                  1hr            625
                  1/2 hour       135
                  1hour          118
                                ... 
                  50-1 hour        1
                  \1 hr-           1
                  ha 9944          1
                  heavy flow       1
                  hr1              1
Name: lowercase, Length: 123, dtype: int64

In [ ]:
delay_df['delay_minutes'] 

In [26]:
def time_unit_replace(text):
  for key in time_unit_map:
    if text.match(key):
      text = text.replace(key, time_unit_map[key])
  return text 

delay_df['time_unit_replace'] = delay_df['lowercase'].apply(time_unit_replace)
delay_df[500005:500015]

AttributeError: ignored

In [163]:
# Replace letter "h" with a " hours"
def add_hspace(text):
  return text.replace('h|hr', ' hours ')

delay_df['time_unit_replaced'] = delay_df['lower_case'].apply(add_hspace)
delay_df[155:165]

,how_long_delayed,lower_case,time_unit_replaced,list_length
155,20,20,20,0
156,15minutes,15minutes,15minutes,1
157,30,30,30,0
158,20,20,20,0
159,1425,1425,1425,0
160,15 min,15 min,15 min,1
161,NaN,nan,nan,0
162,15,15,15,0
163,30 min,30 min,30 min,1
164,45Min,45min,45min,1


In [167]:
# Replace letter "m" with a " m"
def add_mspace(text):
  return text.replace('m', ' minutes ')

delay_df['time_unit_replaced'] = delay_df['time_unit_replaced'].apply(add_mspace)
delay_df[155:165]

,how_long_delayed,lower_case,time_unit_replaced,list_length
155,20,20,20,0
156,15minutes,15minutes,15 minutes inutes,1
157,30,30,30,0
158,20,20,20,0
159,1425,1425,1425,0
160,15 min,15 min,15 minutes in,1
161,NaN,nan,nan,0
162,15,15,15,0
163,30 min,30 min,30 minutes in,1
164,45Min,45min,45 minutes in,1


In [161]:
def get_list_length(text):
  WORD_TOKEN_PATTERN = r'minutes|hours'
  return len(re.findall(WORD_TOKEN_PATTERN, text))
  
delay_df['list_length'] = delay_df['time_unit_replaced'].apply(get_list_length)
delay_df[300055:300065]

,how_long_delayed,lower_case,time_unit_replaced,list_length
300055,31-45 Min,31-45 min,31-45 minutes in,1
300056,46-60 Min,46-60 min,46-60 minutes in,1
300057,16-30 Min,16-30 min,16-30 minutes in,1
300058,46-60 Min,46-60 min,46-60 minutes in,1
300059,31-45 Min,31-45 min,31-45 minutes in,1
300060,NaN,nan,nan,0
300061,16-30 Min,16-30 min,16-30 minutes in,1
300062,46-60 Min,46-60 min,46-60 minutes in,1
300063,0-15 Min,0-15 min,0-15 minutes in,1
300064,NaN,nan,nan,0


In [158]:
delay_df['list_length'].value_counts(dropna=False)

1    466335
0     63056
2       304
Name: list_length, dtype: int64

In [159]:
delay_df[delay_df['list_length'] == 2]

,how_long_delayed,lower_case,time_unit_replaced,list_length
336,1hr/30min,1hr/30min,1 hours r/30 minutes in,2
1170,45min/1hr,45min/1hr,45 minutes in/1 hours r,2
1296,45mini/1hr,45mini/1hr,45 minutes ini/1 hours r,2
2083,45min/1hr,45min/1hr,45 minutes in/1 hours r,2
3170,35min/45mi,35min/45mi,35 minutes in/45 minutes i,2
...,...,...,...,...
301232,1 hr 30min,1 hr 30min,1 hours r 30 minutes in,2
301244,1 hr 30min,1 hr 30min,1 hours r 30 minutes in,2
302615,1 hr 30 mi,1 hr 30 mi,1 hours r 30 minutes i,2
311794,30M MIN,30m min,30 minutes minutes in,2


In [116]:
def get_time_minute(text):
  WORD_TOKEN_PATTERN = r'minutes|hours'
  list_length = len(re.findall(WORD_TOKEN_PATTERN, text))
  if list_length > 0:
    return re.findall(WORD_TOKEN_PATTERN, text)[list_length - 1]  

time_unit = delay_text.apply(get_time_minute)
time_unit

Occurred_On
2015-11-05 08:10:00       None
2015-11-05 08:10:00    minutes
2015-11-05 08:09:00    minutes
2015-11-05 08:12:00    minutes
2015-11-05 08:13:00       None
                        ...   
2022-10-27 07:34:00    minutes
2022-10-31 07:38:00    minutes
2022-11-03 06:30:00       None
2022-11-04 06:03:00    minutes
2022-11-04 05:45:00    minutes
Name: how_long_delayed, Length: 529695, dtype: object

In [117]:
time_unit.value_counts(dropna=False)

minutes    462540
NaN         63056
hours        4099
Name: how_long_delayed, dtype: int64

In [118]:
# Correct spelling of minutes
for key in time_unit.value_counts(dropna=False).index:
  print(key)

minutes
nan
hours


In [93]:
for item in time_unit.value_counts(dropna=False).index:
  replace(item, ' minutes ')

SyntaxError: ignored